In [2]:
import pandas as pd
import os
import unidecode

In [6]:
# the column G_EDAD has different names in the different years, so we need to rename it
def rename_column(df):
    if 'GRANDES_GRUPOS_DE_EDAD' in df.columns:
        df.rename(columns={'GRANDES_GRUPOS_DE_EDAD': 'G_EDAD'}, inplace=True)
        
    elif 'GRANDES_GRUPOS_DE_EDAD_1V' in df.columns:
        df.rename(columns={'GRANDES_GRUPOS_DE_EDAD_1V': 'G_EDAD'}, inplace=True)
    elif 'G_EDAD' in df.columns:
        pass
    else:
        print('G_EDAD column not found in the dataframe')
        return None
    return df
def rename_sexo_column(df):
    if 'SEXO' in df.columns:
        pass
    elif 'JUNTA_SEXO' in df.columns:
        df.rename(columns={'JUNTA_SEXO': 'SEXO'}, inplace=True)
    return df

In [4]:
print(os.getcwd())

d:\Open-ELEC\tests


In [7]:
def recolect_column_values(input_folder, file_name, column_name):
    df_info = pd.DataFrame()
    df_info["ANIO"]=''
    df_info[column_name]=''
    unique_values = {}  # Diccionario para almacenar los valores únicos de la columna y el año correspondiente
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".csv"):
                if file.startswith(file_name):
                    file_path = os.path.join(root, file)
                    anio = file.split(file_name)[1].split(".")[0][1:]
                    #print(anio)
                    df = pd.read_csv(file_path, low_memory=False)
                    df = rename_column(df)
                    df = rename_sexo_column(df)
                    #extract the unique values of the column 
                    uniq_val=df[column_name].unique()
                    #print(uniq_val)
                    #create a dataframe with the unique values of the column
                    df_unique = pd.DataFrame(uniq_val, columns=[column_name])
                    df_unique['ANIO'] = anio
                    df_unique["present"] = 1
                    df_info = pd.concat([df_info, df_unique], ignore_index=True)
    return df_info, unique_values

def crear_df_info(df_info, column_to_study, column_to_use):
    '''
    Crea un data frame con el nombre de la columna que se quiere recuperar el código y ámbito geográfico.
    Y se van a crear columnas para cada año con el código que le corresponde a esa columna en ese año.
    Si no hay código para esa columna en ese año, se va a colocar un NaN
    
    Paramaters
    ----------
        - df_info: DataFrame
            DataFrame con los códigos de la columna que se quiere recuperar el código
        - column_to_study: str
            nombre de la columna que se está estudiando
        - column_to_use: str
            nombre de la columna que se quiere recuperar el código
    
    Returns
    -------
        - df_info_std: DataFrame
            DataFrame con el nombre de la columna que se quiere recuperar el código y ámbito geográfico y columnas para cada año con el código que le corresponde a esa columna en ese año
    
    Examples
    --------
    crear_df_info(df_info, ["DIGNIDAD_NOMBRE", "DIGNIDAD_AMBITO"], "DIGNIDAD_CODIGO")
         
    '''
    # Crear un DataFrame vacío
    df_info_std = pd.DataFrame()
    # Iterar sobre las filas del DataFrame
    for index, row in df_info.iterrows():
        #Print something to check that it is running
        print(row["ANIO"])
        # Revisar si la columna que se quiere recuperar el código ya está en el dataframe
        if row[column_to_study] not in df_info_std.index:
            # Si la columna que se quiere recuperar el código no está en el dataframe, agregarla y colocar su código en la columna correspondiente al año de la elección
            df_info_std.loc[row[column_to_study], row["ANIO"]] = row[column_to_use]
        else:
            # Si la columna que se quiere recuperar el código ya está en el dataframe, colocar su código en la columna correspondiente al año de la elección
            df_info_std.loc[row[column_to_study], row["ANIO"]] = row[column_to_use]
    
    # Colocar las columnas de los años en orden
    df_info_std = df_info_std.reindex(sorted(df_info_std.columns), axis=1)
    # Colocar las filas en orden
    df_info_std = df_info_std.sort_index()
    return df_info_std

    

# Ensure that recolect_column_values and save_column_values are functions and not modules
df,unique_values = recolect_column_values('../data_csv', 'registro_electoral', 'SEXO')
print(df)
df.to_csv('../src/open_elec/data/Codigos_estandar/registro_electoral/SEXO.csv', index=False)
df_info_std = crear_df_info(df, 'SEXO', 'present')
df_info_std.to_csv('../src/open_elec/data/Codigos_estandar/registro_electoral/SEXO_std.csv', index=True)
#save_column_values(unique_values, "../src/open_elec/data/Codigos_estandar/registro_electoral/", 'G_EDAD')


     ANIO       SEXO  present
0    2018    HOMBRES      1.0
1    2018    MUJERES      1.0
2    2002    HOMBRES      1.0
3    2002    MUJERES      1.0
4    2006    HOMBRES      1.0
5    2006    MUJERES      1.0
6    2007    HOMBRES      1.0
7    2007    MUJERES      1.0
8    2009    HOMBRES      1.0
9    2009    MUJERES      1.0
10   2013    HOMBRES      1.0
11   2013    MUJERES      1.0
12   2017    HOMBRES      1.0
13   2017    MUJERES      1.0
14   2021   FEMENINO      1.0
15   2021  MASCULINO      1.0
16   2023  MASCULINO      1.0
17   2023   FEMENINO      1.0
18   2004    HOMBRES      1.0
19   2004    MUJERES      1.0
20   2009    HOMBRES      1.0
21   2009    MUJERES      1.0
22   2014    HOMBRES      1.0
23   2014    MUJERES      1.0
24   2019   FEMENINO      1.0
25   2019  MASCULINO      1.0
26  2023s  MASCULINO      1.0
27  2023s   FEMENINO      1.0
2018
2018
2002
2002
2006
2006
2007
2007
2009
2009
2013
2013
2017
2017
2021
2021
2023
2023
2004
2004
2009
2009
2014
2014
2019
2019


In [8]:
df_electores_name = pd.read_csv('../src/open_elec/data/Codigos_estandar/registro_electoral/electores_names_std.csv')
df_electores_name

,NOMBRE ESTANDARIZADO,ALTERNATIVA 1,ALTERNATIVA 2,ALTERNATIVA 3,ALTERNATIVA 4
0,ELECTORES MENORES A 18,16 A 18 AÑOS,DE 16 A MENOS DE 18 AÑOS,DE 16 a 18 AÑOS,MENOR A 18 AÑOS
1,ELECTORES DE 18 A 65,DE 18 A MENOS DE 65 AÑOS,MAS DE 18 A MENOS DE 65 AÑOS,MAYOR O IGUAL A 18 Y MENOR A 65 AÑOS,MÁS DE 18 A 65 AÑOS
2,ELECTORES MAYORES A 65,DE 65 AÑOS Y MAS,DE 65 AÑOS Y MÁS,MAYOR O IGUAL A 65 AÑOS,MÁS DE 65 AÑOS


In [10]:
df_sexo_names = pd.read_csv('../src/open_elec/data/Codigos_estandar/registro_electoral/sexo_names_std.csv')
df_sexo_names

,NOMBRE ESTANDARIZADO,ALTERNATIVA 1,ALTERNATIVA 2
0,0,MASCULINO,HOMBRES
1,1,FEMENINO,MUJERES
